In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# ---------------------------------------------------------------------------
# 1. FUNÇÕES DE ANÁLISE (MODELO DE MERTON)
# ---------------------------------------------------------------------------

def calcular_risco_individual(df_portfolio, taxa_livre_risco, tempo_anos=1.0):
    """
    Calcula o risco individual (PD, LGD, EL) para cada empresa na carteira
    usando o Modelo de Merton.
    """
    # Copia para evitar modificar o original
    df = df_portfolio.copy()

    # Parâmetros do Modelo de Merton
    A = df['Ativo (MM)']
    L = df['Passivo (MM)']  # Ponto de default (strike price)
    sigma_A = df['Volatilidade dos Ativos']
    r = taxa_livre_risco
    T = tempo_anos

    # Pergunta 1: Calcular Probabilidade de Default (PD)
    # Distância até o Default (DD)
    # DD = (ln(A/L) + (r - 0.5 * sigma_A^2) * T) / (sigma_A * sqrt(T))
    # Usamos 'r' como o drift (retorno esperado dos ativos) - Ver Hipótese 2
    numerador = np.log(A / L) + (r - 0.5 * sigma_A**2) * T
    denominador = sigma_A * np.sqrt(T)
    df['Distancia_Default'] = numerador / denominador

    # A Probabilidade de Default (PD) é N(-DD)
    df['PD_1_Ano'] = norm.cdf(-df['Distancia_Default'])

    # Pergunta 2: Calcular Perda Esperada (EL) e Provisão
    # LGD (Loss Given Default) = Exposição * (1 - Taxa de Recuperação)
    df['LGD (MM)'] = df['Exposicao - EAD (MM)'] * (1 - df['Taxa de Recuperacao'])
    
    # EL (Expected Loss) = PD * LGD
    df['EL (MM)'] = df['PD_1_Ano'] * df['LGD (MM)']
    
    return df

def simular_portfolio(df_risco, taxa_livre_risco, corr_matrix, n_simulacoes, tempo_anos=1.0):
    """
    Executa uma simulação de Monte Carlo para a carteira para encontrar
    a distribuição de perdas e o Value at Risk (VaR).
    """
    
    # Extrair vetores de dados
    n_empresas = len(df_risco)
    A0 = df_risco['Ativo (MM)'].values
    L = df_risco['Passivo (MM)'].values
    sigma_A = df_risco['Volatilidade dos Ativos'].values
    lgd = df_risco['LGD (MM)'].values
    r = taxa_livre_risco
    T = tempo_anos

    # Decomposição de Cholesky da matriz de correlação
    L_cholesky = np.linalg.cholesky(corr_matrix)

    # Gerar variáveis aleatórias normais padrão correlacionadas
    Z_uncorr = np.random.normal(0.0, 1.0, (n_simulacoes, n_empresas))
    Z_corr = Z_uncorr @ L_cholesky.T

    # Calcular o valor terminal dos ativos (A_T) para todas as simulações
    # A_T = A0 * exp( (r - 0.5 * sigma_A^2)T + sigma_A * sqrt(T) * Z )
    drift = (r - 0.5 * sigma_A**2) * T
    shock = (sigma_A * np.sqrt(T) * Z_corr)
    
    # Usamos np.newaxis para alinhar os vetores para broadcasting
    A_T = A0[np.newaxis, :] * np.exp(drift[np.newaxis, :] + shock)

    # Verificar defaults: A_T < L
    defaults = A_T < L[np.newaxis, :]
    
    # Calcular a perda total do portfólio para cada simulação
    # Perda = soma(default_i * LGD_i)
    perdas_simuladas = (defaults * lgd[np.newaxis, :]).sum(axis=1)
    
    return perdas_simuladas

# ---------------------------------------------------------------------------
# 2. CONFIGURAÇÃO DOS DADOS E PARÂMETROS
# ---------------------------------------------------------------------------

# Dados da Carteira
data = {
    'Cliente': ['Empresa A', 'Empresa B', 'Empresa C', 'Empresa D', 'Empresa E'],
    'Setor': ['Varejo', 'Indústria', 'Tecnologia', 'Energia', 'Transporte'],
    'Ativo (MM)': [120, 50, 80, 90, 40],
    'Passivo (MM)': [80, 40, 60, 70, 30],
    'Exposicao - EAD (MM)': [60, 35, 55, 65, 25],
    'Volatilidade dos Ativos': [0.30, 0.40, 0.35, 0.25, 0.50],
    'Taxa de Recuperacao': [0.40, 0.50, 0.60, 0.30, 0.20]
}
df_carteira_base = pd.DataFrame(data).set_index('Cliente')

# Parâmetros Econômicos
TAXA_SELIC = 0.1475
TEMPO_ANOS = 1.0
N_SIMULACOES = 100000

# Hipótese 3: Matriz de Correlação dos Ativos
# Na prática, isso seria estimado. Aqui, assumimos uma correlação
# uniforme de 0.3 entre todas as empresas.
n_empresas = len(df_carteira_base)
CORRELACAO_BASE = np.full((n_empresas, n_empresas), 0.3)
np.fill_diagonal(CORRELACAO_BASE, 1.0)

# ---------------------------------------------------------------------------
# 3. EXECUÇÃO DA ANÁLISE E RESPOSTAS AO CONSELHO
# ---------------------------------------------------------------------------

print("="*80)
print(" ANÁLISE DE RISCO DE CRÉDITO DA CARTEIRA CORPORATIVA (CENÁRIO BASE)")
print("="*80)

# --- Respostas às Perguntas 1 e 2 (Risco Individual) ---
df_risco_base = calcular_risco_individual(df_carteira_base, TAXA_SELIC, TEMPO_ANOS)

print("\n--- PERGUNTA 1: Qual a chance de default (PD) em 1 ano? ---\n")
print(df_risco_base[['Setor', 'PD_1_Ano']].to_string(float_format="{:.2%}".format))

print("\n\n--- PERGUNTA 2: Qual a perda esperada (EL) e a provisão necessária? ---\n")
print(df_risco_base[['Exposicao - EAD (MM)', 'LGD (MM)', 'EL (MM)']].to_string(float_format="{:,.2f}".format))

total_ead = df_risco_base['Exposicao - EAD (MM)'].sum()
total_provisao_el = df_risco_base['EL (MM)'].sum()
significancia = total_provisao_el / total_ead

print("\nResumo da Provisão (Cenário Base):")
print(f"  - Exposição Total (EAD):    {total_ead:,.2f} MM")
print(f"  - Provisão Total (Soma EL): {total_provisao_el:,.2f} MM")
print(f"  - Significância (Provisão/EAD): {significancia:.2%}")
print("\nConclusão Q2: A provisão de Perda Esperada (EL) é significativa, "
      f"correspondendo a {significancia:.2%} da exposição total.")

# --- Resposta à Pergunta 3 (Risco do Portfólio e Concentração) ---
print("\n\n--- PERGUNTA 3: Como as perdas se comportam em conjunto (simulação)? ---\n")
perdas_base = simular_portfolio(df_risco_base, TAXA_SELIC, CORRELACAO_BASE, N_SIMULACOES, TEMPO_ANOS)

# Calculando VaR (Value at Risk) e ES (Expected Shortfall)
var_95 = np.percentile(perdas_base, 95)
var_99 = np.percentile(perdas_base, 99)
es_99 = perdas_base[perdas_base >= var_99].mean()
perda_max = perdas_base.max()
perda_media = perdas_base.mean()

print("Estatísticas da Distribuição de Perdas (Base):")
print(f"  - Perda Média (simulada):   {perda_media:,.2f} MM (próximo da Provisão EL)")
print(f"  - Value at Risk (VaR) 95%:  {var_95:,.2f} MM")
print(f"  - Value at Risk (VaR) 99%:  {var_99:,.2f} MM")
print(f"  - Expected Shortfall (ES) 99%:{es_99:,.2f} MM")
print(f"  - Perda Máxima (em {N_SIMULACOES} sim.): {perda_max:,.2f} MM")

print("\nConclusão Q3: A Perda Esperada (média) é de "
      f"{perda_media:,.2f} MM. No entanto, em 5% dos cenários, a perda excede "
      f"{var_95:,.2f} MM, e em 1% dos cenários, excede {var_99:,.2f} MM. "
      "O capital para 'Perda Inesperada' (UL) deve cobrir a diferença "
      f"entre o VaR e a Perda Esperada (EL), ex: {var_99 - total_provisao_el:,.2f} MM.")

# --- Resposta à Pergunta 4 (Análise de Estresse) ---
print("\n\n" + "="*80)
print(" ANÁLISE DE ESTRESSE: AUMENTO DE 50% NA VOLATILIDADE DOS ATIVOS")
print("="*80)

# Hipótese 4: Cenário de estresse = +50% de aumento relativo na volatilidade
df_carteira_estresse = df_carteira_base.copy()
df_carteira_estresse['Volatilidade dos Ativos'] = df_carteira_estresse['Volatilidade dos Ativos'] * 1.5

# Recalcular Risco Individual (Q1 e Q2) no estresse
df_risco_estresse = calcular_risco_individual(df_carteira_estresse, TAXA_SELIC, TEMPO_ANOS)

print("\n--- Q1 e Q2 (Estresse): PDs e Provisão em Cenário de Estresse ---\n")
print(df_risco_estresse[['PD_1_Ano', 'EL (MM)']].to_string(float_format="{:.2%}".format))

total_ead_estresse = df_risco_estresse['Exposicao - EAD (MM)'].sum()
total_provisao_estresse = df_risco_estresse['EL (MM)'].sum()
significancia_estresse = total_provisao_estresse / total_ead_estresse

print("\nResumo da Provisão (Cenário Estresse):")
print(f"  - Exposição Total (EAD):    {total_ead_estresse:,.2f} MM")
print(f"  - Provisão Total (Soma EL): {total_provisao_estresse:,.2f} MM")
print(f"  - Significância (Provisão/EAD): {significancia_estresse:.2%}")

# Recalcular Risco do Portfólio (Q3) no estresse
perdas_estresse = simular_portfolio(df_risco_estresse, TAXA_SELIC, CORRELACAO_BASE, N_SIMULACOES, TEMPO_ANOS)

var_99_estresse = np.percentile(perdas_estresse, 99)
es_99_estresse = perdas_estresse[perdas_estresse >= var_99_estresse].mean()
perda_media_estresse = perdas_estresse.mean()

print("\n--- Q3 (Estresse): Estatísticas da Distribuição de Perdas ---\n")
print("Estatísticas da Distribuição de Perdas (Estresse):")
print(f"  - Perda Média (simulada):   {perda_media_estresse:,.2f} MM")
print(f"  - Value at Risk (VaR) 99%:  {var_99_estresse:,.2f} MM")
print(f"  - Expected Shortfall (ES) 99%:{es_99_estresse:,.2f} MM")


print("\n\n--- PERGUNTA 4: Comparativo do Impacto Macroeconômico (Volatilidade) ---\n")
print(f"  - Aumento na Provisão (EL):      de {total_provisao_el:,.2f} MM para {total_provisao_estresse:,.2f} MM "
      f"(+{(total_provisao_estresse/total_provisao_el - 1):.1%})")
print(f"  - Aumento no VaR (99%):          de {var_99:,.2f} MM para {var_99_estresse:,.2f} MM "
      f"(+{(var_99_estresse/var_99 - 1):.1%})")
print(f"  - Aumento na Perda Inesperada*:  de {(var_99 - total_provisao_el):,.2f} MM para {(var_99_estresse - total_provisao_estresse):,.2f} MM "
      f"(+{( (var_99_estresse - total_provisao_estresse)/(var_99 - total_provisao_el) - 1):.1%})")
print("  * (VaR 99% - Provisão EL)")

print("\nConclusão Q4: Um aumento de 50% na volatilidade dos ativos tem um impacto "
      "severo e não linear no risco. A provisão de Perda Esperada (EL) "
      f"aumenta em {(total_provisao_estresse/total_provisao_el - 1):.1%}, "
      "e a Perda Inesperada (VaR 99% - EL) necessária como capital de risco "
      f"aumenta {( (var_99_estresse - total_provisao_estresse)/(var_99 - total_provisao_el) - 1):.1%}.")

# ---------------------------------------------------------------------------
# 4. HIPÓTESES ASSUMIDAS NA ANÁLISE
# ---------------------------------------------------------------------------

print("\n\n" + "="*80)
print(" HIPÓTESES E PREMISSAS UTILIZADAS NA ANÁLISE")
print("="*80)
print("1. Modelo de Merton: A análise assume que o Modelo de Merton é adequado. "
      "O default ocorre se o valor dos Ativos (A) cair abaixo do Passivo (L) no vencimento de 1 ano.")
print("2. Retorno dos Ativos (Drift): Assumimos que o retorno esperado dos ativos (μ) "
      "de todas as empresas é igual à taxa livre de risco (Selic = 14.75%). Esta é uma "
      "suposição conservadora; um retorno esperado maior reduziria as PDs.")
print("3. Ponto de Default: Assumimos que o valor 'Passivo (MM)' representa o ponto de "
      "default (face value da dívida) em 1 ano.")
print("4. Correlação: Assumimos uma matriz de correlação de ativos uniforme e constante de 0.3 "
      "para todas as empresas. Em uma análise real, esta matriz seria estimada "
      "empiricamente a partir de dados de mercado (ex: correlação de ações).")
print("5. Cenário de Estresse: O cenário adverso foi definido como um aumento relativo "
      "de 50% na volatilidade dos ativos de todas as empresas (ex: 30% -> 45%).")

 ANÁLISE DE RISCO DE CRÉDITO DA CARTEIRA CORPORATIVA (CENÁRIO BASE)

--- PERGUNTA 1: Qual a chance de default (PD) em 1 ano? ---

                Setor  PD_1_Ano
Cliente                        
Empresa A      Varejo     4.52%
Empresa B   Indústria    23.37%
Empresa C  Tecnologia    14.27%
Empresa D     Energia     7.07%
Empresa E  Transporte    26.75%


--- PERGUNTA 2: Qual a perda esperada (EL) e a provisão necessária? ---

           Exposicao - EAD (MM)  LGD (MM)  EL (MM)
Cliente                                           
Empresa A                    60     36.00     1.63
Empresa B                    35     17.50     4.09
Empresa C                    55     22.00     3.14
Empresa D                    65     45.50     3.22
Empresa E                    25     20.00     5.35

Resumo da Provisão (Cenário Base):
  - Exposição Total (EAD):    240.00 MM
  - Provisão Total (Soma EL): 17.43 MM
  - Significância (Provisão/EAD): 7.26%

Conclusão Q2: A provisão de Perda Esperada (EL) é signific